# Custom models with `simple`

The easiest way to create a `simple` model is to simply call `simple.Model` with your custom likelihood function.
However, there are many cases where one could want a model with custom methods, pre-defined parameters, etc.
This can be done through custom model classes.

To demonstrate this, we re-implement the celerite model from the [Gaussian Processes tutorial](./gaussian_processes.ipynb) using a custom class.

In [1]:
import numpy as np
from celerite2 import GaussianProcess, terms

from simple import Model


class CustomModel(Model):
    def __init__(self, parameters, t, y, yerr, initial_parameters: dict | None):
        super().__init__(parameters, self._log_likelihood)
        self.t = t
        self.y = y
        self.yerr = yerr

        if initial_parameters is None:
            initial_parameters = self.get_prior_samples(1)
        self.gp = GaussianProcess(self.get_kernel(initial_parameters))
        self.gp.compute(self.t, yerr=self.yerr)

    def get_kernel(self, params: dict):
        return terms.SHOTerm(
            sigma=params["sigma_per"], rho=params["rho_per"], tau=params["tau_per"]
        ) + terms.SHOTerm(sigma=params["sigma_non"], rho=params["rho_non"], Q=0.25)

    def set_params(self, params: dict):
        self.gp.mean = params["mean"]
        self.gp.kernel = self.get_kernel(params)
        self.gp.compute(self.t, diag=self.yerr**2 + params["sigma2"], quiet=True)
        return self.gp

    def _log_likelihood(self, parameters: dict) -> float:
        self.gp = self.set_params(parameters)
        return self.gp.log_likelihood(self.y)

In [2]:
t = np.sort(
    np.append(
        np.random.uniform(0, 3.8, 57),
        np.random.uniform(5.5, 10, 68),
    )
)  # The input coordinates must be sorted
yerr = np.random.uniform(0.08, 0.22, len(t))
y = 0.2 * (t - 5) + np.sin(3 * t + 0.1 * (t - 5) ** 2) + yerr * np.random.randn(len(t))

In [3]:
from scipy.stats import norm

from simple.distributions import ScipyDistribution

prior_dist = norm(0, 2)
parameters = {
    "mean": ScipyDistribution(prior_dist),
    "sigma_per": ScipyDistribution(prior_dist),
    "rho_per": ScipyDistribution(prior_dist),
    "tau_per": ScipyDistribution(prior_dist),
    "sigma_non": ScipyDistribution(prior_dist),
    "rho_non": ScipyDistribution(prior_dist),
    "sigma2": ScipyDistribution(prior_dist),
}
initial_params = [0.0, 1.0, 1.0, 10.0, 1.0, 5.0, 0.01]
model = CustomModel(
    parameters,
    t,
    y,
    yerr,
    initial_parameters=dict(zip(parameters.keys(), initial_params, strict=False)),
)
model.log_likelihood(initial_params)

np.float64(-19.784859422234867)